# Context managers

## BProf Python course

### June 25-29, 2018

#### Judit Ács

## Managed resources

- resource acquisition and release are automatically done
- no need for manual resource management
- example: memory
  - C++ has both managed and unmanaged memory management. The stack is managed, but the heap is not, we need to manually call `new` and `delete`.

## Unmanaged resources

- unmanaged resources need explicit release
- otherwise the operating system may run out of the resource
- examples include files, network sockets

In [1]:
fh = []
while True:
    try:
        fh.append(open("abc.txt", "w"))
    except OSError:
        break
len(fh)

970

In [2]:
for f in fh:
    f.close()

- we need to manually close the file
- what happens when an exception occurs

In [3]:
s1 = "important text"
fh = open("file.txt", "w")
# fh.write(s2)  # raises NameError
fh.close()

- the file is never closed, the file descriptor **is leaked**
- a solution would be to use try-except blocks with `finally` clauses

In [4]:
from sys import stderr

fh = open("file.txt", "w")
try:
    fh.write(important_variable)
except Exception as e:
    stderr.write("{0} happened".format(type(e).__name__))
finally:
    print("Closing file")
    fh.close()

Closing file


NameError happened

## Context managers handle this automatically

- the `with` keyword opens a resource
- keeps it open until the execution leaves with's scope
- releases the resource regardless whether an exception is raised or not

In [5]:
with open("file.txt", "w") as fh:
    fh.write("abc\n")
    # fh.write(important_variable)  # raises NameError

## Defining context managers

- any class can be a context manager if it implements:
  1. `__enter__`: runs at the beginning of the `with`. Returns the resource.
  1. `__exit__`: runs after the with block. Releases the resource.

In [6]:
class DummyContextManager:
    def __init__(self, value):
        self.value = value
        
    def __enter__(self):
        print("Dummy resource acquired")
        return self.value
    
    def __exit__(self, *args):
        print("Dummy resource released")
        
with DummyContextManager(42) as d:
    print("Resource: {}".format(d))

Dummy resource acquired
Resource: 42
Dummy resource released


In [7]:
d = DummyContextManager("abc")
print("DummyContextManager created")

with d as resource:
    print("Resource", resource)

DummyContextManager created
Dummy resource acquired
Resource abc
Dummy resource released


`__exit__` takes 3 extra arguments that describe the exception: `exc_type`, `exc_value`, `traceback`

In [8]:
from sys import stderr

class DummyContextManager:
    def __init__(self, value):
        self.value = value
        
    def __enter__(self):
        print("Dummy resource acquired")
        return self.value
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            stderr.write("{0} with value {1} caught\n"
                  "Traceback: {2}".format(
                      exc_type, exc_value, traceback))
        print("Dummy resource released")
        
with DummyContextManager(42) as d:
    print(d)
    # raise ValueError("just because I can")  # __exit__ will be called anyway

Dummy resource acquired
42
Dummy resource released


In [9]:
with DummyContextManager(42) as d:
    print(d)
    raise ValueError("just because I can")  # __exit__ will be called anyway

Dummy resource acquired
42
Dummy resource released


<class 'ValueError'> with value just because I can caught
Traceback: <traceback object at 0x7fb888523148>

ValueError: just because I can

## Using multiple context managers in the same block

In [ ]:
with open("/tmp/dmesg") as infile, open("/tmp/dmesg.copy", "w") as outfile:
    for line in infile:
        if "0.00" not in line:
            outfile.write(line)